# Packages needed to access data

In [1]:
import pandas as pd
import numpy as np

# Retrieve QUBO matrix of an instance
- How to retrieve the QUBO matrix of instance `id`?
- Which problem class is `id` instance of?

In [2]:
# Select instance with id = 5
# By changing id, we can retrieve the QUBO matrix of other instances
id = 5
Q = pd.read_csv(f'data/qubo_dataset/{id}.csv', index_col=[0]).to_numpy()

# The characteristics of the instance are stored in file map_index_to_instance.csv
map_df = pd.read_csv('data/map_index_to_instance.csv', index_col=[0])
print(map_df.loc[id])

problem          NumPart
structure      geometric
n_variables           27
repetition           0.0
Name: 5, dtype: object


# Solve an instance with Quantum Annealing
We want to solve the instance 5 with quantum annealing.
To do this, you need the `ocean-sdk`.
To do so, we proceed with the following steps:
1. Transform matrix `Q` into a Binary Quadratic Problem (BQM)
2. Embed the BQM onto the quantum annealer
3. Sample solutions

We need the following packages:

In [3]:
import dimod
from dwave.system import DWaveSampler, FixedEmbeddingComposite
from ast import literal_eval

In the case we want to use a precomputed embedding, as one of those in the `embeddings` folder, we rely on these helper functions we have defined.

In [5]:
from dwave.system import FixedEmbeddingComposite
from ast import literal_eval

# Return a dictionary containing the embedding of the instance.
def extract_embedding(id):
    dataframe = pd.read_csv(f'data/embeddings/{id}.csv', index_col=0, header=None, dtype=object)
    dataframe[1] = dataframe[1].apply(lambda x: literal_eval(x))
    dictionary = dataframe.to_dict()
    return dictionary[1]

embedding = extract_embedding(id)
print(embedding)

{0: [436, 3241, 435, 3242], 1: [510, 2987, 511, 2986], 2: [3182, 3180, 481, 3181], 3: [331, 330, 3286], 4: [3137, 3136, 3138, 556], 5: [585, 586, 3062, 3061], 6: [376, 377, 375], 7: [391, 392, 3226, 390], 8: [2972, 2971, 525, 526], 9: [3302, 3376, 420, 332, 421], 10: [3197, 3196, 3195, 240], 11: [480, 3031, 3030, 255], 12: [495, 3016, 3015, 496], 13: [3092, 3091, 615], 14: [3256, 316, 3257, 315], 15: [3152, 3153, 3151, 661], 16: [345, 3001, 346, 347], 17: [406, 405, 407, 3211], 18: [361, 360, 362], 19: [3121, 3122, 450], 20: [3046, 3045, 630, 3047], 21: [3076, 3077, 600], 22: [3436, 302, 301, 300], 23: [3316, 286, 3317, 285], 24: [271, 3271, 270, 3272], 25: [3168, 3166, 3167], 26: [465, 3106, 3107]}


In [6]:
def array_to_dict(a):
    n = len(a)
    d = {}
    for i in range(n):
        for j in range(i, n):
            if a[i, j] != 0:
                d[(i, j)] = a[i, j]
    return d

Q_dict = array_to_dict(Q)
print(Q_dict)

{(0, 0): -38496.0, (0, 1): 6848.0, (0, 2): 4928.0, (0, 3): 8320.0, (0, 4): 896.0, (0, 5): 768.0, (0, 6): 1728.0, (0, 7): 5760.0, (0, 8): 1472.0, (0, 9): 9600.0, (0, 10): 640.0, (0, 11): 2240.0, (0, 12): 2048.0, (0, 13): 576.0, (0, 14): 4928.0, (0, 15): 4032.0, (0, 16): 640.0, (0, 17): 1152.0, (0, 18): 2880.0, (0, 19): 640.0, (0, 20): 1216.0, (0, 21): 3968.0, (0, 22): 3968.0, (0, 23): 2432.0, (0, 24): 4224.0, (0, 25): 320.0, (0, 26): 768.0, (1, 1): -472512.0, (1, 2): 65912.0, (1, 3): 111280.0, (1, 4): 11984.0, (1, 5): 10272.0, (1, 6): 23112.0, (1, 7): 77040.0, (1, 8): 19688.0, (1, 9): 128400.0, (1, 10): 8560.0, (1, 11): 29960.0, (1, 12): 27392.0, (1, 13): 7704.0, (1, 14): 65912.0, (1, 15): 53928.0, (1, 16): 8560.0, (1, 17): 15408.0, (1, 18): 38520.0, (1, 19): 8560.0, (1, 20): 16264.0, (1, 21): 53072.0, (1, 22): 53072.0, (1, 23): 32528.0, (1, 24): 56496.0, (1, 25): 4280.0, (1, 26): 10272.0, (2, 2): -349272.0, (2, 3): 80080.0, (2, 4): 8624.0, (2, 5): 7392.0, (2, 6): 16632.0, (2, 7): 55440

1. Transform the QUBO matrix into BQM format

In [7]:
bqm = dimod.binary.BinaryQuadraticModel.from_qubo(Q_dict)

2. Embed the problem on DWave Advantage6.4. You need a token to access the Quantum Annealer.

In [8]:
qpu = DWaveSampler(token='d-wave_access_token',
                   solver={'name': 'Advantage_system6.4'})

sampler = FixedEmbeddingComposite(qpu, embedding)

SolverAuthenticationError: Invalid token or access denied

3. Solve the instances with quantum annealing. We define the number of samples `num_reads` and the `annealing_time` (microseconds)

In [9]:
num_reads = 50
annealing_time = 20
sampleset = sampler.sample(bqm,
                           num_reads=num_reads,
                           annealing_time=annealing_time)

NameError: name 'sampler' is not defined

We can solve the QUBO problem also with another solver, as Simulated Annealing

In [10]:
from dwave.samplers import SimulatedAnnealingSampler

num_reads = 50
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads=num_reads)

# Sample with the best value of the cost function
first = sampleset.first
print(first)

Sample(sample={0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1, 6: 0, 7: 1, 8: 0, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 0, 25: 0, 26: 1}, energy=-1466520.0, num_occurrences=1)


# Meta-Learning Dataset 

In [11]:
metalearning_df = pd.read_csv('data/metalearning_dataset.csv', 
                              index_col=[0], header=[0,1])

# Results of the Meta-models
We access to the results of the meta-models trained over the small instances.
Change `size_instances` to `'large'` to see the results for the large instances.


In [13]:
size_instances = 'small'
metamodels_results_df = pd.read_csv(f'data/metamodels_results/{size_instances}/metamodels_results.csv',
                                    index_col=[0,1],
                                    header=[0,1,2])
metamodels_results_df

SolSpace                                          \
                     random_forest                               ada_boost   
                 balanced_accuracy balanced_accuracy_std balanced_accuracy   
Optimal       QA          0.831689              0.041702          0.931043   
              SA          0.798153              0.042767          0.883321   
              TS          0.562393              0.124786          0.842185   
              SD          0.635526              0.166419          0.932313   
1e-05-Optimal QA          0.808595              0.052460          0.901260   
              SA          0.761394              0.145934          0.831602   
              TS          0.500000              0.000000          0.674394   
              SD          0.500000              0.000000          0.995652   
h-Optimal     QA          0.849755              0.058660          0.912513   
              SA          0.627381              0.160056          0.859777   
              TS          0.555556              0.111111          0.760435   
              SD          0.571429              0.142857          0.871288   
QA_over_all   QA          0.771145              0.144465          0.892312   

                                                          \
                                                xg_boost   
                 balanced_accuracy_std balanced_accuracy   
Optimal       QA              0.021284          0.780131   
              SA              0.064577          0.889369   
              TS              0.016849          0.821154   
              SD              0.047173          0.887946   
1e-05-Optimal QA              0.061130          0.888051   
              SA              0.110641          0.848864   
              TS              0.048110          0.709141   
              SD              0.008696          0.581159   
h-Optimal     QA              0.039050          0.915962   
              SA              0.080269          0.858711   
              TS              0.053745          0.784178   
              SD              0.045756          0.875833   
QA_over_all   QA              0.015488          0.849080   

                                                            \
                                       logistic_regression   
                 balanced_accuracy_std   balanced_accuracy   
Optimal       QA              0.131359            0.767226   
              SA              0.058073            0.572443   
              TS              0.038550            0.551099   
              SD              0.044423            0.553788   
1e-05-Optimal QA              0.069972            0.739523   
              SA              0.058090            0.500000   
              TS              0.137541            0.500000   
              SD              0.103768            0.500000   
h-Optimal     QA              0.037280            0.568867   
              SA              0.067179            0.529125   
              TS              0.073078            0.562500   
              SD              0.047230            0.585714   
QA_over_all   QA              0.086994            0.768470   

                                                  NorMul  \
                                           random_forest   
                 balanced_accuracy_std balanced_accuracy   
Optimal       QA              0.056834          0.714251   
              SA              0.041964          0.622869   
              TS              0.070355          0.664766   
              SD              0.088231          0.644158   
1e-05-Optimal QA              0.114090          0.686531   
              SA              0.000000          0.531176   
              TS              0.000000          0.500000   
              SD              0.000000          0.500000   
h-Optimal     QA              0.031252          0.722727   
              SA              0.033097          0.611199   
              TS              0.096825     

# Training and testing a meta-model
We give an example on how to train and test a meta-model with the Meta-Learning dataset.

We train an AdaBoost meta-model with the `LogIsing` domain of the small instances, to predict the label `Optimal` for the Quantum Annealing (`QA`). We use balanced accuracy to evaluate the performance of the meta-model.
Some features of this domain are not defined for some instances.
In our work, we substitute the `nan` values of a certain feature `f` with the mean value of `f`. We do it also here. 

In [14]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import MinMaxScaler

# Features in the domain LogIsing
domain = 'LogIsing'
X = metalearning_df.loc[range(246), domain]

# Target data
target = ('Optimal', 'QA')
y = metalearning_df.loc[range(246), target]

# Replace not defined value in X with the mean values of the features
a = np.isinf(X)
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(X.mean(), inplace=True)

# Features are scaled in the range [0,1]
scaler = MinMaxScaler()
for col in X.columns:
    transformed_col = scaler.fit_transform(pd.DataFrame(X.loc[:,col]))
    X.loc[:,col] = transformed_col

We generate two data splits: one contains the data used for the training (67% of the instances), the other contains the data used to test the meta-model (33% of the instances). In this example, we generate the stratified splits over the problem class.

In [15]:
from sklearn.model_selection import train_test_split

test_size = 0.33
problem = ('instance_id', 'problem')

X_train, X_test, y_train, y_test = train_test_split(X, y.values, test_size=test_size, stratify= metalearning_df.loc[range(246), problem])

Now we train the model and compute its balanced_accuracy on the test split.

In [16]:
# Training of the meta-model
model = AdaBoostClassifier()
model.fit(X_train, y_train)

# Testing of the meta-model
y_predicted = model.predict(X_test)
balanced_accuracy = balanced_accuracy_score(y_test, y_predicted)
print(f'Balanced Accuracy of the meta-model: {balanced_accuracy}')

Balanced Accuracy of the meta-model: 0.8266908212560387


We can compute the importance of the features of the metamodel with permutation feature importance.
Each feature is shuffled for `n_repeats = 100` times and the loss in the balanced accuracy is computed.

In [17]:
from sklearn.inspection import permutation_importance

permutation_importances = permutation_importance(model, X_test, y_test,
                                                scoring='balanced_accuracy', n_repeats=100)

In [18]:
feature_importances = {}
for feature, importance  in zip(X.columns, permutation_importances.importances_mean):
    feature_importances[feature] = importance
    
# We sort the feature from the most important one to the least important one
print(f'FEATURE IMPORTANCES')
for key in sorted(feature_importances, key=feature_importances.get, reverse=True):
    print(f'{key}: {feature_importances[key]}')

FEATURE IMPORTANCES
Laplacian_spectral_gap: 0.06277777777777777
Structural Degree_hhi: 0.04172101449275362
Adjacency_radius: 0.041588164251207746
Degree_gini_index: 0.03736714975845411
Bias_gini_index: 0.035078502415458956
Structural Degree_max_eigval: 0.021841787439613524
Structural Adjacency_min_eigval: 0.019516908212560385
Bias_condition_number: 0.015072463768115938
Adjacency_shannon_entropy: 0.011962560386473427
Adjacency_gini_index: 0.010797101449275396
Normalized Laplacian_hhi: 0.010223429951690826
Normalized Laplacian_shannon_entropy: 0.009196859903381644
Adjacency_hhi: 0.007210144927536212
Structural Adjacency_gini_index: 0.005138888888888871
Adjacency_spectral_gap: 0.0048188405797101345
Laplacian_connectivity: 0.0044565217391304545
Structural Degree_gini_index: 0.004426328502415472
Bias_min_eigval: 0.003369565217391307
Laplacian_gini_index: 0.002971014492753643
Degree_hhi: 0.00028381642512077864
Adjacency_condition_number: 0.0
Structural Adjacency_condition_number: 0.0
Normali

# Feature Importance
We access to the results of the feature importance of the meta-models trained over the small instances.
Change `size_instances` to `'large'` to see the results for the large instances.

In [19]:
size_instances = 'small'
feature_importance_df = pd.read_csv(f'data/feature_importance/{size_instances}/feature_importance.csv',
                                    index_col=[0,1],
                                    header=[0,1,2])
feature_importance_df

Optimal            \
                                                                QA             
                                                     random_forest ada_boost   
SolSpace  gini_index                                      0.205036  0.172687   
          hhi                                             0.000000  0.021672   
          grouped_hhi                                     0.003976  0.221262   
          shannon_entropy                                 0.000839  0.046280   
          min                                            -0.002288  0.034504   
...                                                            ...       ...   
StructLap EMBEDDING ISING GRAPH_hhi                       0.000000  0.001480   
          EMBEDDING ISING GRAPH_shannon_entropy           0.000000  0.000000   
          EMBEDDING ISING GRAPH_spectral_gap              0.000000  0.015843   
          EMBEDDING ISING GRAPH_connectivity              0.000000  0.019567   
          EMBEDDING ISING GRAPH_connected_components      0.000000  0.000000   

                                                                \
                                                                 
                                                      xg_boost   
SolSpace  gini_index                                  0.127895   
          hhi                                         0.005900   
          grouped_hhi                                 0.081225   
          shannon_entropy                            -0.010177   
          min                                         0.000886   
...                                                        ...   
StructLap EMBEDDING ISING GRAPH_hhi                  -0.001739   
          EMBEDDING ISING GRAPH_shannon_entropy       0.016614   
          EMBEDDING ISING GRAPH_spectral_gap          0.008837   
          EMBEDDING ISING GRAPH_connectivity          0.008939   
          EMBEDDING ISING GRAPH_connected_components  0.000000   

                                                                          \
                                                                           
                                                     logistic_regression   
SolSpace  gini_index                                            0.075184   
          hhi                                                  -0.002777   
          grouped_hhi                                           0.182249   
          shannon_entropy                                       0.000353   
          min                                                   0.000040   
...                                                                  ...   
StructLap EMBEDDING ISING GRAPH_hhi                             0.000341   
          EMBEDDING ISING GRAPH_shannon_entropy                 0.004654   
          EMBEDDING ISING GRAPH_spectral_gap                    0.000000   
          EMBEDDING ISING GRAPH_connectivity                    0.025019   
          EMBEDDING ISING GRAPH_connected_components            0.001884   

                                                                              \
                                                                SA             
                                                     random_forest ada_boost   
SolSpace  gini_index                                      0.190974  0.074814   
          hhi                                             0.002667  0.042535   
          grouped_hhi                                     0.085735  0.190312   
          shannon_entropy                                -0.000316  0.047055   
          min                                             0.019628  0.034718   
...                                                            ...       ...   
StructLap EMBEDDING ISING GRAPH_hhi                       0.002490  0.000000   
          EMBEDDING ISING GRAPH_shannon_entropy           0.004282  0.000000   
          EMBEDDING ISING GRAPH_spectral_gap              0.010546  